# Approach for 3D coordinates

In [ ]:
import os
import json
import numpy as np
from zipfile import ZipFile

In [ ]:
#from files
from walkDir import walkDir

In [ ]:
def get_ids(path, folder):
    frames = walkDirZip(path = path, interested_folder = folder)
    frames = [frame.split("/")[-1] for frame in frames]
    ids = [frame.split("_")[-1].split(".")[0].replace('hd', '') for frame in frames]
    ids.sort()
    return ids


def get_frames(path):
    pose = 'hdPose3d_stage1_op25/'
    hands = 'hdHand3d/'
    frames_pose = walkDir(dname = path + pose, filt = r"\.json$")
    frames_hands = walkDir(dname = path + hands, filt = r"\.json$")
    frames_pose.sort()
    frames_hands.sort()
    return frames_pose, frames_hands

def selectPointsPose(points, keepThis):
    points2 = []
    for i in keepThis:
        points2.append(points[4 * i + 0])
        points2.append(points[4 * i + 1])
        points2.append(points[4 * i + 2])
        confidence_score = max(0, points[4 * i + 3]) #Set confidence values of -1 to 0
        points2.append(confidence_score)
    return points2

def selectPointsHands(points, confidence, keepThis):
    points2 = []
    for i in keepThis:
        points2.append(points[3 * i + 0])
        points2.append(points[3 * i + 1])
        points2.append(points[3 * i + 2])
        confidence_score = max(0, confidence[i]) #Set confidence values of -1 to 0
        points2.append(confidence_score)
        if points2[-1] == -1:
            print("Ai mareta")
    return points2

def noNones(l):
    l2 = []
    for i in l:
        if not i is None:
            l2.append(i)
    return l2

def walkDirZip(path, interested_folder):
    with ZipFile(path, 'r') as zip:
        list_dir = zip.namelist()
    frames = []
    for file in list_dir:
        path = file.split('/')
        if path[1] in interested_folder and len(path) > 2 and path[-1][-4:] == 'json':
            frames.append(file)
    
    return frames


def get_frames_zip(path):
    pose = 'hdPose3d_stage1_op25'
    hands = 'hdHand3d'
    frames_pose = walkDirZip(path = path, interested_folder = pose)
    frames_hands = walkDirZip(path = path, interested_folder = hands)
    frames_pose.sort()
    frames_hands.sort()
    return frames_pose, frames_hands

In [ ]:
def loadData(dname):
    
    pose_ids = get_ids(dname, 'hdPose3d_stage1_op25/')
    hands_ids = get_ids(dname, 'hdHand3d/')
    
    #if pose_ids != hands_ids:
    #    raise ValueError(f"Number of frames of pose and hands do not match {len(pose_ids)}, {len(hands_ids)}")
        
    pose_fnames, hands_fnames = get_frames_zip(dname)
    
    frames = [None for _ in range(len(pose_fnames))]
    #for i, (framepose, framehands) in enumerate(zip(pose_fnames, hands_fnames)):
    i = 0
    idx_pose = 0
    idx_hand = 0
    with ZipFile(dname, 'r') as zip:
        while idx_pose < len(pose_fnames) and idx_hand < len(hands_fnames):
            if pose_ids[idx_pose] == hands_ids[idx_hand]:
                data_pose = json.loads(zip.read(pose_fnames[idx_pose]))
                #with open(framepose) as json_data:
                #    data_pose = json.load(json_data)

                data_hands = json.loads(zip.read(hands_fnames[idx_hand]))
                #with open(framehands) as json_data:
                #    data_hands = json.load(json_data)

                #read body coordinates
                pointsP = None
                for person in data_pose['bodies']:
                    if str(person['id']) == '0':
                        pointsP = np.array(person['joints26'])

                #read hands coordinates
                pointsLH = None
                confidenceLH = None
                pointsRH = None
                confidenceRH = None
                for person in data_hands['people']:
                    if str(person['id']) == '0':
                        if 'left_hand' in person.keys():
                            pointsLH = np.array(person['left_hand']['landmarks'])
                            confidenceLH = np.array(person['left_hand']['averageScore'])

                        if 'right_hand' in person.keys():
                            pointsRH = np.array(person['right_hand']['landmarks'])
                            confidenceRH = np.array(person['right_hand']['averageScore'])

                # idxsPose = [0, 1, 2, 3, 4, 5, 6, 7]
                # idxsPose = [1, 0, 3, 4, 5, 9, 10, 11]
                idxsPose = range(25)
                idxsHand = range(21)

                if pointsP is None:
                    pointsP = 4 * 25 * [0.0]
                if pointsLH is None:
                    pointsLH = 4 * len(idxsHand) * [0.0]
                if pointsRH is None:
                    pointsRH = 4 * len(idxsHand) * [0.0]
                if confidenceLH is None:
                    confidenceLH = len(idxsHand) * [0.0]
                if confidenceRH is None:
                    confidenceRH = len(idxsHand) * [0.0]


                pointsP = selectPointsPose(pointsP, idxsPose)
                pointsLH = selectPointsHands(pointsLH, confidenceLH, idxsHand)
                pointsRH = selectPointsHands(pointsRH, confidenceRH, idxsHand)
                points = pointsP + pointsRH + pointsLH

                if not points[0] == 0.0:
                    frames[i] = points

                i += 1
                idx_pose += 1
                idx_hand += 1

            elif pose_ids[idx_pose] < hands_ids[idx_hand]:
                #print("misalignment", pose_ids[idx_pose], hands_ids[idx_hand], idx_pose, idx_hand)
                idx_pose += 1

            else:
                #print("misalignment", pose_ids[idx_pose], hands_ids[idx_hand], idx_pose, idx_hand)
                idx_hand += 1
        

    return np.asarray(noNones(frames), dtype="float32")

In [ ]:
data_dir = "/mnt/cephfs/How2Sign/panoptic_data/"
zip_folders = [file for file in os.listdir(data_dir) if file[-4:] == '.zip'] #'190425_asl10.zip'

output_path = "../Data/preprocessed_01/"

for zip_folder in zip_folders:
    print("Loading ", zip_folder)
    data_01 = loadData(data_dir + zip_folder)

    current_file = zip_folder[:-4]
    with open(output_path + '/' + current_file + '.npy', 'wb') as f:
        np.save(f, data_01)
        
    print("Done!")

In [ ]:
fnameIn = "../Data/preprocessed_01/"
with open(fnameIn + "190425_asl10.npy", 'rb') as f:
    a = np.load(f)

In [ ]:
len(a[0])

In [ ]:
data_01.shape

In [ ]:
date_dir = "/mnt/cephfs/How2Sign/panoptic_data/"

zip_folders = [file for file in os.listdir(date_dir) if file[-3:] == 'zip'] #'190425_asl10.zip'

### Additional txt file "lrSwitchedFrames"

In [ ]:
!cat ~/clean_data/190611_asl2/hdHand3d/lrSwitchedFrames.txt

In [ ]:
!cat ~/clean_data/190611_asl2/hdHand3d/handRecon3D_hd00008099.json